In [65]:
import sys
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np
import sys
import math
import csv

In [66]:
num_nearest_neighbors = 300#k (num_nearest_neighbors) chosen [100, 300, 500, 700, 900, 1100,1300]
within_decay_rate = 0.9# chosen from [0.1, 0.2, 0.3,0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
group_decay_rate = 0.7# chosen from [0.1, 0.2, 0.3,0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
alpha = 0.7
group_size = 7
topk = 10

#files = ['history.csv', 'future.csv']

files = ['TaFang_history_NB.csv', 'TaFang_future_NB.csv']
attributes_list = 'MATERIAL_NUMBER'
usr_attr = 'CUSTOMER_ID'
ord_attr = 'ORDER_NUMBER'

In [67]:
next_k_step = 1
freq_max = 200
## all the follow three ways array. First index is patient, second index is the time step, third is the feature vector
data_chunk = []
day_gap_counter = []
claims_counter = 0
num_claim = 0
activate_codes_num = -1
training_chunk = 0
test_chunk = 1

df1=pd.read_csv(files[0])
tmp1=df1.sort_values([usr_attr,ord_attr]).groupby([usr_attr,ord_attr])[attributes_list].apply(np.array).reset_index()
tmp1=tmp1.groupby([usr_attr])[attributes_list].apply(list).reset_index()
tmp1[attributes_list]=tmp1[attributes_list].apply(lambda x: [[-1]]+[np.array(sorted(i)) for i in x]+[[-1]])

df2=pd.read_csv(files[1])
tmp2=df2.sort_values([usr_attr,ord_attr]).groupby([usr_attr,ord_attr])[attributes_list].apply(np.array).reset_index()
tmp2=tmp2.groupby([usr_attr])[attributes_list].apply(list).reset_index()
tmp2[attributes_list]=tmp2[attributes_list].apply(lambda x: [[-1]]+[np.array(sorted(i)) for i in x]+[[-1]])

past_chunk = 0
future_chunk = 1

history_key_set,future_key_set=tmp1,tmp2

history_key_set[attributes_list]=np.where(tmp1[attributes_list].apply(len)<= 3,None,tmp1[attributes_list])
history_key_set[usr_attr]=history_key_set[usr_attr].apply(str)
history_key_set=list(history_key_set[~history_key_set[attributes_list].isnull()][usr_attr])

future_key_set[attributes_list]=np.where(tmp2[attributes_list].apply(len)< 2 + next_k_step,None,tmp2[attributes_list])
future_key_set[usr_attr]=future_key_set[usr_attr].apply(str)
future_key_set=list(future_key_set[~future_key_set[attributes_list].isnull()][usr_attr])

filtered_key_set=list(pd.Series(history_key_set)[pd.Series(history_key_set).isin(future_key_set) & pd.Series(history_key_set).isin(future_key_set).isin(history_key_set)])
#filtered_key_set=[x for x in history_key_set if x in future_key_set]
#slice filtered_key_set range(0:10001)
training_key_set = filtered_key_set[0:int(4 / 5 * len(filtered_key_set)*0.9)]
print('Number of training instances: ' + str(len(training_key_set)))
#slice filtered_key_set range(10001:11112)
validation_key_set = filtered_key_set[int(4 / 5 * len(filtered_key_set)*0.9):int(4 / 5 * len(filtered_key_set))]
#slice filtered_key_set range(:11112)
test_key_set = filtered_key_set[int(4 / 5 * len(filtered_key_set)):]

num_dim=len(set(df1[attributes_list]))
code_freq_at_first_claim = np.zeros(num_dim+2)
data_chunk=[dict(zip(tmp1[usr_attr].apply(str),tmp1[attributes_list])),dict(zip(tmp2[usr_attr].apply(str),tmp2[attributes_list]))]

#data_chunk=data_chunk
input_size=num_dim + 2
#code_freq_at_first_claim=code_freq_at_first_claim 
#data_chunk=data_chunk
key_set=list(data_chunk[test_chunk])
data_set=data_chunk[training_chunk]
key_set=training_key_set 
output_size=input_size
#within_decay_rate=within_decay_rate
#group_decay_rate=group_decay_rate
print('Num. of top: ', topk)

sum_history = {}
for key in key_set:
    vec_list = data_set[key]
    num_vec = len(vec_list)-2
    his_list = []
    
    for idx in range(1,num_vec+1):
        his_vec = np.zeros(output_size)#output_size=11999
        # np.power(.9,15-iterater)  #counts down 
        decayed_val = np.power(within_decay_rate,num_vec-idx)
        #elements in data_set['2']
        for ele in vec_list[idx]:
            his_vec[ele] = decayed_val
        his_list.append(his_vec)
        
    grouped_vec_list = []
    if len(his_list) < group_size:
        #sum = np.zeros(len(his_list[0]))
        for j in range(len(his_list)):
            grouped_vec_list.append(his_list[j])

        grouped_list,real_group_size = grouped_vec_list, len(his_list)
    else:
        est_num_vec_each_block = len(his_list)/group_size
        base_num_vec_each_block = int(np.floor(len(his_list)/group_size))
        residual = est_num_vec_each_block - base_num_vec_each_block

        num_vec_has_extra_vec = int(np.round(residual * group_size))

        if residual == 0:
            for i in range(group_size):
                if len(his_list)<1:
                    print('len(his_list)<1')
                sum = np.zeros(len(his_list[0]))
                for j in range(base_num_vec_each_block):
                    if i*base_num_vec_each_block+j >= len(his_list):
                        print('i*num_vec_each_block+j')
                    sum += his_list[i*base_num_vec_each_block+j]
                grouped_vec_list.append(sum/base_num_vec_each_block)
        else:
            for i in range(group_size - num_vec_has_extra_vec):
                sum = np.zeros(len(his_list[0]))
                for j in range(base_num_vec_each_block):
                    if i*base_num_vec_each_block+j >= len(his_list):
                        print('i*base_num_vec_each_block+j')
                    sum += his_list[i*base_num_vec_each_block+j]
                    last_idx = i * base_num_vec_each_block + j
                grouped_vec_list.append(sum/base_num_vec_each_block)

            est_num = int(np.ceil(est_num_vec_each_block))
            start_group_idx = group_size - num_vec_has_extra_vec
            if len(his_list) - start_group_idx*base_num_vec_each_block >= est_num_vec_each_block:
                for i in range(start_group_idx,group_size):
                    sum = np.zeros(len(his_list[0]))
                    for j in range(est_num):
                        # if residual+(i-1)*est_num_vec_each_block+j >= len(his_list):
                        #     print('residual+(i-1)*num_vec_each_block+j')
                        #     print('len(his_list)')
                        iidxx = last_idx + 1+(i-start_group_idx)*est_num+j
                        if  iidxx >= len(his_list) or iidxx<0:
                            print('last_idx + 1+(i-start_group_idx)*est_num+j')
                        sum += his_list[iidxx]
                    grouped_vec_list.append(sum/est_num)

        grouped_list,real_group_size = grouped_vec_list, group_size

    his_vec = np.zeros(output_size)
    for idx in range(real_group_size):
        decayed_val = np.power(group_decay_rate, group_size - 1 - idx)
        if idx>=len(grouped_list):
            print( 'idx: '+ str(idx))
            print('len(grouped_list): ' + str(len(grouped_list)))
        his_vec += grouped_list[idx]*decayed_val
    sum_history[key] = his_vec/real_group_size
    # sum_history[key] = np.multiply(his_vec / real_group_size, IDF)

temporal_decay_sum_history_training = sum_history

data_set=data_chunk[training_chunk] 
key_set=test_key_set 
output_size=input_size
#group_size=group_size
#within_decay_rate=within_decay_rate
#group_decay_rate=group_decay_rate
#data_set, key_set, output_size,group_size,within_decay_rate,group_decay_rate=data_chunk[training_chunk],test_key_set, input_size,group_size, within_decay_rate,group_decay_rate

sum_history = {}
for key in key_set:
    vec_list = data_set[key]
    num_vec = len(vec_list) - 2
    his_list = []
    
        #removes the [-1] flags in data
    for idx in range(1,num_vec+1):
        his_vec = np.zeros(output_size)#output_size=11999
        # np.power(.9,15-iterater)  #counts down 
        decayed_val = np.power(within_decay_rate,num_vec-idx)
        #elements in data_set['2']
        for ele in vec_list[idx]:
            his_vec[ele] = decayed_val
        his_list.append(his_vec)
        
    grouped_vec_list = []
    if len(his_list) < group_size:
        #sum = np.zeros(len(his_list[0]))
        for j in range(len(his_list)):
            grouped_vec_list.append(his_list[j])

        grouped_list,real_group_size = grouped_vec_list, len(his_list)
    else:
        est_num_vec_each_block = len(his_list)/group_size
        base_num_vec_each_block = int(np.floor(len(his_list)/group_size))
        residual = est_num_vec_each_block - base_num_vec_each_block

        num_vec_has_extra_vec = int(np.round(residual * group_size))

        if residual == 0:
            for i in range(group_size):
                if len(his_list)<1:
                    print('len(his_list)<1')
                sum = np.zeros(len(his_list[0]))
                for j in range(base_num_vec_each_block):
                    if i*base_num_vec_each_block+j >= len(his_list):
                        print('i*num_vec_each_block+j')
                    sum += his_list[i*base_num_vec_each_block+j]
                grouped_vec_list.append(sum/base_num_vec_each_block)
        else:
            for i in range(group_size - num_vec_has_extra_vec):
                sum = np.zeros(len(his_list[0]))
                for j in range(base_num_vec_each_block):
                    if i*base_num_vec_each_block+j >= len(his_list):
                        print('i*base_num_vec_each_block+j')
                    sum += his_list[i*base_num_vec_each_block+j]
                    last_idx = i * base_num_vec_each_block + j
                grouped_vec_list.append(sum/base_num_vec_each_block)

            est_num = int(np.ceil(est_num_vec_each_block))
            start_group_idx = group_size - num_vec_has_extra_vec
            if len(his_list) - start_group_idx*base_num_vec_each_block >= est_num_vec_each_block:
                for i in range(start_group_idx,group_size):
                    sum = np.zeros(len(his_list[0]))
                    for j in range(est_num):
                        # if residual+(i-1)*est_num_vec_each_block+j >= len(his_list):
                        #     print('residual+(i-1)*num_vec_each_block+j')
                        #     print('len(his_list)')
                        iidxx = last_idx + 1+(i-start_group_idx)*est_num+j
                        if  iidxx >= len(his_list) or iidxx<0:
                            print('last_idx + 1+(i-start_group_idx)*est_num+j')
                        sum += his_list[iidxx]
                    grouped_vec_list.append(sum/est_num)

        grouped_list,real_group_size = grouped_vec_list, group_size
    his_vec = np.zeros(output_size)
    for idx in range(real_group_size):
        decayed_val = np.power(group_decay_rate, group_size - 1 - idx)
        if idx>=len(grouped_list):
            print( 'idx: '+ str(idx))
            print('len(grouped_list): ' + str(len(grouped_list)))
        his_vec += grouped_list[idx]*decayed_val
    sum_history[key] = his_vec/real_group_size
    # sum_history[key] = np.multiply(his_vec / real_group_size, IDF)
    
temporal_decay_sum_history_test = sum_history
query_set=temporal_decay_sum_history_test 
target_set=temporal_decay_sum_history_training 
k=num_nearest_neighbors

#def KNN(query_set, target_set, k):
history_mat = []
for key in target_set.keys():
    history_mat.append(target_set[key])
test_mat = []
for key in query_set.keys():
    test_mat.append(query_set[key])
print('Finding k nearest neighbors...')
nbrs = NearestNeighbors(n_neighbors=k, algorithm='brute').fit(history_mat)
distances, indices = nbrs.kneighbors(test_mat)
print('Finish KNN search.' )
index, distance = indices,distances

sum_history_test,test_key_set,training_sum_history_test,training_key_set,index,alpha = temporal_decay_sum_history_test, test_key_set, temporal_decay_sum_history_training,training_key_set, index, alpha
merged_history = {}
for test_key_id in range(len(test_key_set)):
    test_key = test_key_set[test_key_id]
    test_history = sum_history_test[test_key]
    sum_training_history = np.zeros(len(test_history))
    for indecis in index[test_key_id]:
        training_key = training_key_set[indecis]
        sum_training_history += training_sum_history_test[training_key]

    sum_training_history = sum_training_history/len(index[test_key_id])

    merge = test_history*alpha + sum_training_history*(1-alpha)
    merged_history[test_key] = merge

sum_history = merged_history

if activate_codes_num < 0:
    # for i in range(1, 6):
    prec = []
    rec = []
    F = []
    prec1 = []
    rec1 = []
    F1 = []
    prec2 = []
    rec2 = []
    F2 = []
    prec3 = []
    rec3 = []
    F3 = []
    NDCG = []
    n_hit = 0

    num_ele = topk
    # print('k = ' + str(activate_codes_num))
    # evaluate(data_chunk, input_size,test_KNN_history, test_key_set, next_k_step)
    count = 0
    for iter in range(len(test_key_set)):
        
        # training_pair = training_pairs[iter - 1]
        # input_variable = training_pair[0]
        # target_variable = training_pair[1]
        input_variable = data_chunk[training_chunk][test_key_set[iter]]
        target_variable = data_chunk[test_chunk][test_key_set[iter]]

        if len(target_variable) < 2 + next_k_step:
            continue
        count += 1
        sum_history,key = sum_history, test_key_set[iter]
        #def predict_with_elements_in_input(sum_history,key):
        output_vectors = []

        for idx in range(next_k_step):
            vec = sum_history[key]
            output_vectors.append(vec)
        output_vectors = output_vectors        
        top = 400
        hit = 0
        for idx in range(len(output_vectors)):
            # for idx in [2]:

            output = np.zeros(input_size)
            target_topi = output_vectors[idx].argsort()[::-1][:top]
            c = 0
            for i in range(top):
                if c >= num_ele:
                    break
                output[target_topi[i]] = 1
                c += 1

            vectorized_target = np.zeros(input_size)
            for ii in target_variable[1 + idx]:
                vectorized_target[ii] = 1
            groundtruth,pred = vectorized_target, output
            #def get_precision_recall_Fscore(groundtruth,pred):
            a = groundtruth
            b = pred
            correct = 0
            truth = 0
            positive = 0

            for idx in range(len(a)):
                if a[idx] == 1:
                    truth += 1
                    if b[idx] == 1:
                        correct += 1
                if b[idx] == 1:
                    positive += 1

            flag = 0
            if 0 == positive:
                precision = 0
                flag = 1
                #print('postivie is 0')
            else:
                precision = correct/positive
            if 0 == truth:
                recall = 0
                flag = 1
                #print('recall is 0')
            else:
                recall = correct/truth

            if flag == 0 and precision + recall > 0:
                G = 2*precision*recall/(precision+recall)
            else:
                G = 0
            precision, recall, Fscore, correct = precision, recall, G, correct            
            prec.append(precision)
            rec.append(recall)
            F.append(Fscore)
            if idx == 0:
                prec1.append(precision)
                rec1.append(recall)
                F1.append(Fscore)
            elif idx == 1:
                prec2.append(precision)
                rec2.append(recall)
                F2.append(Fscore)
            elif idx == 2:
                prec3.append(precision)
                rec3.append(recall)
                F3.append(Fscore)
            groundtruth, pred_rank_list,k=vectorized_target, target_topi, num_ele
            #def get_HT(groundtruth, pred_rank_list,k):
            count = 0
            for pred in pred_rank_list:
                if count >= k:
                    break
                if groundtruth[pred] == 1:
                    hit += 1
                count += 1
            hit += 0
            #input_size = 100
            vectorized_target, target_topi, num_ele=groundtruth, pred_rank_list,k
            #def get_NDCG1(groundtruth, pred_rank_list,k):
            count = 0
            dcg = 0
            for pred in pred_rank_list:
                if count >= k:
                    break
                if groundtruth[pred] == 1:
                    dcg += (1)/math.log2(count+1+1)
                count += 1
            idcg = 0
            num_real_item = np.sum(groundtruth)
            num_item = int(num_real_item)
            for i in range(num_item):
                idcg += (1) / math.log2(i + 1 + 1)
            ndcg = dcg / idcg
            NDCG.append(ndcg)
        if hit == next_k_step:
            n_hit += 1
            
    print('average precision of ' + ': ' + str(np.mean(prec)) + ' with std: ' + str(np.std(prec)))
    recall = np.mean(rec)
    ndcg = np.mean(NDCG)
    hr = n_hit / len(test_key_set)
    
recall, ndcg, hr = recall, ndcg, hr
print('recall: ', str(recall))
print('NDCG: ', str(ndcg))

Number of training instances: 10001
Num. of top:  10
Finding k nearest neighbors...
Finish KNN search.
average precision of : 0.053364519611371 with std: 0.07979832834575636
recall:  0.1296420382929529
NDCG:  0.09875611529474698


In [1]:
import sys
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np
import sys
import math
import csv

sys.maxsize
sys.maxsize

num_nearest_neighbors = 300#k (num_nearest_neighbors) chosen [100, 300, 500, 700, 900, 1100,1300]
within_decay_rate = 0.9# chosen from [0.1, 0.2, 0.3,0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
group_decay_rate = 0.7# chosen from [0.1, 0.2, 0.3,0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
alpha = 0.7
group_size = 7
topk = 10

next_k_step = 1
freq_max = 200
## all the follow three ways array. First index is patient, second index is the time step, third is the feature vector
data_chunk = []
day_gap_counter = []
claims_counter = 0
num_claim = 0
activate_codes_num = -1
training_chunk = 0
test_chunk = 1

files = ['TaFang_history_NB.csv', 'TaFang_future_NB.csv']
attributes_list = 'MATERIAL_NUMBER'
usr_attr = 'CUSTOMER_ID'
ord_attr = 'ORDER_NUMBER'

In [2]:
df1=pd.read_csv(files[0])
tmp1=df1.sort_values([usr_attr,ord_attr]).groupby([usr_attr,ord_attr])[attributes_list].apply(np.array).reset_index()
tmp1=tmp1.groupby([usr_attr])[attributes_list].apply(list).reset_index()
tmp1[attributes_list]=tmp1[attributes_list].apply(lambda x: [[-1]]+[np.array(sorted(i)) for i in x]+[[-1]])

In [3]:
df2=pd.read_csv(files[1])
tmp2=df2.sort_values([usr_attr,ord_attr]).groupby([usr_attr,ord_attr])[attributes_list].apply(np.array).reset_index()
tmp2=tmp2.groupby([usr_attr])[attributes_list].apply(list).reset_index()
tmp2[attributes_list]=tmp2[attributes_list].apply(lambda x: [[-1]]+[np.array(sorted(i)) for i in x]+[[-1]])

In [4]:
past_chunk = 0
future_chunk = 1
history_key_set,future_key_set=tmp1,tmp2

history_key_set[attributes_list]=np.where(tmp1[attributes_list].apply(len)<= 3,None,tmp1[attributes_list])
history_key_set[usr_attr]=history_key_set[usr_attr].apply(str)
history_key_set=list(history_key_set[~history_key_set[attributes_list].isnull()][usr_attr])

future_key_set[attributes_list]=np.where(tmp2[attributes_list].apply(len)< 2 + next_k_step,None,tmp2[attributes_list])
future_key_set[usr_attr]=future_key_set[usr_attr].apply(str)
future_key_set=list(future_key_set[~future_key_set[attributes_list].isnull()][usr_attr])

filtered_key_set=list(pd.Series(history_key_set)[pd.Series(history_key_set).isin(future_key_set) & pd.Series(history_key_set).isin(future_key_set).isin(history_key_set)])
#filtered_key_set=[x for x in history_key_set if x in future_key_set]
#slice filtered_key_set range(0:10001)
training_key_set = filtered_key_set[0:int(4 / 5 * len(filtered_key_set)*0.9)]
print('Number of training instances: ' + str(len(training_key_set)))
#slice filtered_key_set range(10001:11112)
validation_key_set = filtered_key_set[int(4 / 5 * len(filtered_key_set)*0.9):int(4 / 5 * len(filtered_key_set))]
#slice filtered_key_set range(:11112)
test_key_set = filtered_key_set[int(4 / 5 * len(filtered_key_set)):]

Number of training instances: 10001


In [5]:
num_dim=len(set(df1[attributes_list]))
code_freq_at_first_claim = np.zeros(num_dim+2)

data_chunk=[dict(zip(tmp1[usr_attr].apply(str),tmp1[attributes_list])),dict(zip(tmp2[usr_attr].apply(str),tmp2[attributes_list]))]

In [6]:
#data_chunk=data_chunk
input_size=num_dim + 2
#code_freq_at_first_claim=code_freq_at_first_claim 
#data_chunk=data_chunk
key_set=list(data_chunk[test_chunk]),
next_k_step=1

In [7]:
data_set=data_chunk[training_chunk]
key_set=training_key_set 
output_size=input_size
#within_decay_rate=within_decay_rate
#group_decay_rate=group_decay_rate
print('Num. of top: ', topk)

Num. of top:  10


In [8]:
sum_history = {}
for key in key_set:
    vec_list = data_set[key]
    num_vec = len(vec_list)-2
    his_list = []
    
    for idx in range(1,num_vec+1):
        his_vec = np.zeros(output_size)#output_size=11999
        # np.power(.9,15-iterater)  #counts down 
        decayed_val = np.power(within_decay_rate,num_vec-idx)
        #elements in data_set['2']
        for ele in vec_list[idx]:
            his_vec[ele] = decayed_val
        his_list.append(his_vec)
        
    grouped_vec_list = []
    if len(his_list) < group_size:
        #sum = np.zeros(len(his_list[0]))
        for j in range(len(his_list)):
            grouped_vec_list.append(his_list[j])

        grouped_list,real_group_size = grouped_vec_list, len(his_list)
    else:
        est_num_vec_each_block = len(his_list)/group_size
        base_num_vec_each_block = int(np.floor(len(his_list)/group_size))
        residual = est_num_vec_each_block - base_num_vec_each_block

        num_vec_has_extra_vec = int(np.round(residual * group_size))

        if residual == 0:
            for i in range(group_size):
                if len(his_list)<1:
                    print('len(his_list)<1')
                sum = np.zeros(len(his_list[0]))
                for j in range(base_num_vec_each_block):
                    if i*base_num_vec_each_block+j >= len(his_list):
                        print('i*num_vec_each_block+j')
                    sum += his_list[i*base_num_vec_each_block+j]
                grouped_vec_list.append(sum/base_num_vec_each_block)
        else:
            for i in range(group_size - num_vec_has_extra_vec):
                sum = np.zeros(len(his_list[0]))
                for j in range(base_num_vec_each_block):
                    if i*base_num_vec_each_block+j >= len(his_list):
                        print('i*base_num_vec_each_block+j')
                    sum += his_list[i*base_num_vec_each_block+j]
                    last_idx = i * base_num_vec_each_block + j
                grouped_vec_list.append(sum/base_num_vec_each_block)

            est_num = int(np.ceil(est_num_vec_each_block))
            start_group_idx = group_size - num_vec_has_extra_vec
            if len(his_list) - start_group_idx*base_num_vec_each_block >= est_num_vec_each_block:
                for i in range(start_group_idx,group_size):
                    sum = np.zeros(len(his_list[0]))
                    for j in range(est_num):
                        # if residual+(i-1)*est_num_vec_each_block+j >= len(his_list):
                        #     print('residual+(i-1)*num_vec_each_block+j')
                        #     print('len(his_list)')
                        iidxx = last_idx + 1+(i-start_group_idx)*est_num+j
                        if  iidxx >= len(his_list) or iidxx<0:
                            print('last_idx + 1+(i-start_group_idx)*est_num+j')
                        sum += his_list[iidxx]
                    grouped_vec_list.append(sum/est_num)

        grouped_list,real_group_size = grouped_vec_list, group_size

    his_vec = np.zeros(output_size)
    for idx in range(real_group_size):
        decayed_val = np.power(group_decay_rate, group_size - 1 - idx)
        if idx>=len(grouped_list):
            print( 'idx: '+ str(idx))
            print('len(grouped_list): ' + str(len(grouped_list)))
        his_vec += grouped_list[idx]*decayed_val
    sum_history[key] = his_vec/real_group_size
    # sum_history[key] = np.multiply(his_vec / real_group_size, IDF)

In [9]:
temporal_decay_sum_history_training = sum_history

In [10]:
data_set=data_chunk[training_chunk] 
key_set=test_key_set 
output_size=input_size
group_size=group_size
within_decay_rate=within_decay_rate
group_decay_rate=group_decay_rate
#data_set, key_set, output_size,group_size,within_decay_rate,group_decay_rate=data_chunk[training_chunk],test_key_set, input_size,group_size, within_decay_rate,group_decay_rate

In [11]:
sum_history = {}
for key in key_set:
    vec_list = data_set[key]
    num_vec = len(vec_list) - 2
    his_list = []
    
    for idx in range(1,num_vec+1):
        his_vec = np.zeros(output_size)#output_size=11999
        # np.power(.9,15-iterater)  #counts down 
        decayed_val = np.power(within_decay_rate,num_vec-idx)
        #elements in data_set['2']
        for ele in vec_list[idx]:
            his_vec[ele] = decayed_val
        his_list.append(his_vec)
        
    grouped_vec_list = []
    if len(his_list) < group_size:
        #sum = np.zeros(len(his_list[0]))
        for j in range(len(his_list)):
            grouped_vec_list.append(his_list[j])

        grouped_list,real_group_size = grouped_vec_list, len(his_list)
    else:
        est_num_vec_each_block = len(his_list)/group_size
        base_num_vec_each_block = int(np.floor(len(his_list)/group_size))
        residual = est_num_vec_each_block - base_num_vec_each_block

        num_vec_has_extra_vec = int(np.round(residual * group_size))

        if residual == 0:
            for i in range(group_size):
                if len(his_list)<1:
                    print('len(his_list)<1')
                sum = np.zeros(len(his_list[0]))
                for j in range(base_num_vec_each_block):
                    if i*base_num_vec_each_block+j >= len(his_list):
                        print('i*num_vec_each_block+j')
                    sum += his_list[i*base_num_vec_each_block+j]
                grouped_vec_list.append(sum/base_num_vec_each_block)
        else:
            for i in range(group_size - num_vec_has_extra_vec):
                sum = np.zeros(len(his_list[0]))
                for j in range(base_num_vec_each_block):
                    if i*base_num_vec_each_block+j >= len(his_list):
                        print('i*base_num_vec_each_block+j')
                    sum += his_list[i*base_num_vec_each_block+j]
                    last_idx = i * base_num_vec_each_block + j
                grouped_vec_list.append(sum/base_num_vec_each_block)

            est_num = int(np.ceil(est_num_vec_each_block))
            start_group_idx = group_size - num_vec_has_extra_vec
            if len(his_list) - start_group_idx*base_num_vec_each_block >= est_num_vec_each_block:
                for i in range(start_group_idx,group_size):
                    sum = np.zeros(len(his_list[0]))
                    for j in range(est_num):
                        # if residual+(i-1)*est_num_vec_each_block+j >= len(his_list):
                        #     print('residual+(i-1)*num_vec_each_block+j')
                        #     print('len(his_list)')
                        iidxx = last_idx + 1+(i-start_group_idx)*est_num+j
                        if  iidxx >= len(his_list) or iidxx<0:
                            print('last_idx + 1+(i-start_group_idx)*est_num+j')
                        sum += his_list[iidxx]
                    grouped_vec_list.append(sum/est_num)

        grouped_list,real_group_size = grouped_vec_list, group_size
    his_vec = np.zeros(output_size)
    for idx in range(real_group_size):
        decayed_val = np.power(group_decay_rate, group_size - 1 - idx)
        if idx>=len(grouped_list):
            print( 'idx: '+ str(idx))
            print('len(grouped_list): ' + str(len(grouped_list)))
        his_vec += grouped_list[idx]*decayed_val
    sum_history[key] = his_vec/real_group_size
    # sum_history[key] = np.multiply(his_vec / real_group_size, IDF)

In [12]:
temporal_decay_sum_history_test = sum_history

In [13]:
query_set=temporal_decay_sum_history_test 
target_set=temporal_decay_sum_history_training 
k=num_nearest_neighbors

In [14]:
#def KNN(query_set, target_set, k):
history_mat = []
for key in target_set.keys():
    history_mat.append(target_set[key])
test_mat = []
for key in query_set.keys():
    test_mat.append(query_set[key])
print('Finding k nearest neighbors...')
nbrs = NearestNeighbors(n_neighbors=k, algorithm='brute').fit(history_mat)
distances, indices = nbrs.kneighbors(test_mat)
print('Finish KNN search.' )
index, distance = indices,distances

Finding k nearest neighbors...
Finish KNN search.


In [15]:
sum_history_test,test_key_set,training_sum_history_test,training_key_set,index,alpha = temporal_decay_sum_history_test, test_key_set, temporal_decay_sum_history_training,training_key_set, index, alpha 

In [16]:
merged_history = {}
for test_key_id in range(len(test_key_set)):
    test_key = test_key_set[test_key_id]
    test_history = sum_history_test[test_key]
    sum_training_history = np.zeros(len(test_history))
    for indecis in index[test_key_id]:
        training_key = training_key_set[indecis]
        sum_training_history += training_sum_history_test[training_key]

    sum_training_history = sum_training_history/len(index[test_key_id])

    merge = test_history*alpha + sum_training_history*(1-alpha)
    merged_history[test_key] = merge
    
sum_history = merged_history

In [17]:
if activate_codes_num < 0:
    # for i in range(1, 6):
    prec = []
    rec = []
    F = []
    prec1 = []
    rec1 = []
    F1 = []
    prec2 = []
    rec2 = []
    F2 = []
    prec3 = []
    rec3 = []
    F3 = []
    NDCG = []
    n_hit = 0

    num_ele = topk
    # print('k = ' + str(activate_codes_num))
    # evaluate(data_chunk, input_size,test_KNN_history, test_key_set, next_k_step)
    count = 0
    for iter in range(len(test_key_set)):
        
        # training_pair = training_pairs[iter - 1]
        # input_variable = training_pair[0]
        # target_variable = training_pair[1]
        input_variable = data_chunk[training_chunk][test_key_set[iter]]
        target_variable = data_chunk[test_chunk][test_key_set[iter]]

        if len(target_variable) < 2 + next_k_step:
            continue
        count += 1
        sum_history,key = sum_history, test_key_set[iter]
        #def predict_with_elements_in_input(sum_history,key):
        output_vectors = []

        for idx in range(next_k_step):
            vec = sum_history[key]
            output_vectors.append(vec)
        output_vectors = output_vectors        
        top = 400
        hit = 0
        for idx in range(len(output_vectors)):
            # for idx in [2]:

            output = np.zeros(input_size)
            target_topi = output_vectors[idx].argsort()[::-1][:top]
            c = 0
            for i in range(top):
                if c >= num_ele:
                    break
                output[target_topi[i]] = 1
                c += 1

            vectorized_target = np.zeros(input_size)
            for ii in target_variable[1 + idx]:
                vectorized_target[ii] = 1
            groundtruth,pred = vectorized_target, output
            #def get_precision_recall_Fscore(groundtruth,pred):
            a = groundtruth
            b = pred
            correct = 0
            truth = 0
            positive = 0

            for idx in range(len(a)):
                if a[idx] == 1:
                    truth += 1
                    if b[idx] == 1:
                        correct += 1
                if b[idx] == 1:
                    positive += 1

            flag = 0
            if 0 == positive:
                precision = 0
                flag = 1
                #print('postivie is 0')
            else:
                precision = correct/positive
            if 0 == truth:
                recall = 0
                flag = 1
                #print('recall is 0')
            else:
                recall = correct/truth

            if flag == 0 and precision + recall > 0:
                G = 2*precision*recall/(precision+recall)
            else:
                G = 0
            precision, recall, Fscore, correct = precision, recall, G, correct            
            prec.append(precision)
            rec.append(recall)
            F.append(Fscore)
            if idx == 0:
                prec1.append(precision)
                rec1.append(recall)
                F1.append(Fscore)
            elif idx == 1:
                prec2.append(precision)
                rec2.append(recall)
                F2.append(Fscore)
            elif idx == 2:
                prec3.append(precision)
                rec3.append(recall)
                F3.append(Fscore)
            groundtruth, pred_rank_list,k=vectorized_target, target_topi, num_ele
            #def get_HT(groundtruth, pred_rank_list,k):
            count = 0
            for pred in pred_rank_list:
                if count >= k:
                    break
                if groundtruth[pred] == 1:
                    hit += 1
                count += 1
            hit += 0
            #input_size = 100
            vectorized_target, target_topi, num_ele=groundtruth, pred_rank_list,k
            #def get_NDCG1(groundtruth, pred_rank_list,k):
            count = 0
            dcg = 0
            for pred in pred_rank_list:
                if count >= k:
                    break
                if groundtruth[pred] == 1:
                    dcg += (1)/math.log2(count+1+1)
                count += 1
            idcg = 0
            num_real_item = np.sum(groundtruth)
            num_item = int(num_real_item)
            for i in range(num_item):
                idcg += (1) / math.log2(i + 1 + 1)
            ndcg = dcg / idcg
            NDCG.append(ndcg)
        if hit == next_k_step:
            n_hit += 1
            
    print('average precision of ' + ': ' + str(np.mean(prec)) + ' with std: ' + str(np.std(prec)))
    recall = np.mean(rec)
    ndcg = np.mean(NDCG)
    hr = n_hit / len(test_key_set)
    
recall, ndcg, hr = recall, ndcg, hr
print('recall: ', str(recall))
print('NDCG: ', str(ndcg))

average precision of : 0.053364519611371 with std: 0.07979832834575636
recall:  0.1296420382929529
NDCG:  0.09875611529474698
